**Mount Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
# Load and preprocess the data
df = pd.read_csv('Heart.csv')
X = df.drop(columns=['target'])
y = df['target']

print(X.shape), print(y.shape)

(303, 13)
(303,)


(None, None)

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [5]:
# Define base models for blending
base_models = {
    'rf': RandomForestClassifier(n_estimators=10, random_state=42),
    'knn': KNeighborsClassifier(n_neighbors=10),
    'gbdt': GradientBoostingClassifier()
}

In [6]:
# Train base models
base_model_predictions = []
for name, model in base_models.items():
    model.fit(X_train, y_train)
    base_model_predictions.append(model.predict_proba(X_test))

In [7]:
# Stack predictions to create the blending dataset
blending_X = np.column_stack([preds[:, 1] for preds in base_model_predictions])
blending_y = y_test

In [8]:
# Train the meta-model on the blending dataset
meta_model = LogisticRegression()
meta_model.fit(blending_X, blending_y)

LogisticRegression()

In [9]:
# Predict on the test set using the blending model
test_base_model_predictions = []
for name, model in base_models.items():
    test_base_model_predictions.append(model.predict_proba(X_test))

test_blending_X = np.column_stack([preds[:, 1] for preds in test_base_model_predictions])
blending_predictions = meta_model.predict(test_blending_X)

In [10]:
# Evaluate the blending model
blending_accuracy = accuracy_score(y_test, blending_predictions)
print(f"Accuracy of Blending Classifier: {blending_accuracy:.4f}")

Accuracy of Blending Classifier: 0.8361


In [11]:
# Initialize stacking classifier
stacking_clf = StackingClassifier(
    estimators=[(name, model) for name, model in base_models.items()],
    final_estimator=LogisticRegression(),
    cv=10
)


In [12]:
# Train and evaluate the stacking classifier
stacking_clf.fit(X_train, y_train)
stacking_predictions = stacking_clf.predict(X_test)
stacking_accuracy = accuracy_score(y_test, stacking_predictions)
print(f"Accuracy of Stacking Classifier: {stacking_accuracy:.4f}")

Accuracy of Stacking Classifier: 0.8689
